In [1]:
import os
import sys

# get the current script's directory
current_dir = os.path.abspath('')

# get the parent directory by going one level up
parent_dir = os.path.dirname(current_dir)

# add the parent directory to sys.path
sys.path.append(parent_dir)

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial import distance

from joblib import Parallel, delayed

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem

In [3]:
df_esol = pd.read_csv('data/ESOL/delaney.csv')
df_esol

,smiles,logSolubility
0,OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)...,-0.770
1,Cc1occc1C(=O)Nc2ccccc2,-3.300
2,CC(C)=CCCC(C)=CC(=O),-2.060
3,c1ccc2c(c1)ccc3c2ccc4c5ccccc5ccc43,-7.870
4,c1ccsc1,-1.330
...,...,...
1123,FC(F)(F)C(Cl)Br,-1.710
1124,CNC(=O)ON=C(SC)C(=O)N(C)C,0.106
1125,CCSCCSP(=S)(OC)OC,-3.091
1126,CCC(C)C,-3.180


In [4]:
df_esol['inchi_key'] = df_esol.smiles.apply(lambda x: Chem.inchi.MolToInchiKey(Chem.MolFromSmiles(x)) if Chem.MolFromSmiles(x) else 'Remove')

In [5]:
df_esol = df_esol.drop_duplicates(subset='inchi_key')
df_esol.shape

(1117, 3)

In [6]:
smiles_list1 = df_esol.smiles.values[:600]
smiles_list2 = df_esol.smiles.values[600:]

# Calculate Tanimoto/Jaccard Similarity with Bit Vectors

These methods agree!

## RDKit

In [7]:
fp_gen = AllChem.GetRDKitFPGenerator(minPath=1, maxPath=2, fpSize=2048)
def get_RDKit_fp(smi):
	mol = Chem.MolFromSmiles(smi)
	return fp_gen.GetFingerprint(mol)

In [8]:
def get_sim(smi1, smiles_list):
	sim_results = np.zeros(len(smiles_list))
	for i, smi2 in enumerate(smiles_list):
		fp1 = get_RDKit_fp(smi1)
		fp2 = get_RDKit_fp(smi2)

		# todo: explore DataStructs.BulkTanimotoSimilarity() as another way to do this
		sim = DataStructs.TanimotoSimilarity(fp1, fp2)
		sim_results[i] = sim

	return sim_results

In [11]:
def get_similarity_matrix(smiles_list1, smiles_list2, ncpus=1):
	"""
	Returns a matrix of Tanimoto similarities as a numpy array
	of size smiles_list1 x smiles_list2.

	Progress is measured by the number of smiles in smiles_list1.
	"""

	result = Parallel(n_jobs=ncpus, backend='multiprocessing', verbose=5)(delayed(get_sim)(smi, smiles_list2) for smi in smiles_list1)

	similarity_matrix = np.zeros((len(smiles_list1), len(smiles_list2)))
	for i, res in enumerate(result):
		similarity_matrix[i, :] = res

	return similarity_matrix

In [13]:
similarity_matrix_rdkit = get_similarity_matrix(smiles_list1, smiles_list2, ncpus=1)
similarity_matrix_rdkit.shape

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    3.9s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    8.4s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   15.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:   24.0s


(600, 517)

In [14]:
similarity_matrix_rdkit

array([[0.13043478, 0.05263158, 0.28571429, ..., 0.09803922, 0.14285714,
        0.23529412],
       [0.2173913 , 0.15789474, 0.25      , ..., 0.01694915, 0.        ,
        0.13559322],
       [0.        , 0.41176471, 0.        , ..., 0.05263158, 0.33333333,
        0.04444444],
       ...,
       [0.14285714, 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.22857143],
       [0.09090909, 0.        , 0.18518519, ..., 0.01960784, 0.        ,
        0.09259259],
       [0.125     , 0.        , 0.25      , ..., 0.        , 0.        ,
        0.20512821]])

## Scipy

In [15]:
fps1 = [get_RDKit_fp(smi) for smi in smiles_list1]
fps2 = [get_RDKit_fp(smi) for smi in smiles_list2]

In [16]:
# this calculates distance so subtrac 1 to make it similarity
similarity_matrix_scipy = 1 - distance.cdist(fps1, fps2, metric='jaccard')
similarity_matrix_scipy.shape

(600, 517)

In [17]:
similarity_matrix_scipy

array([[0.13043478, 0.05263158, 0.28571429, ..., 0.09803922, 0.14285714,
        0.23529412],
       [0.2173913 , 0.15789474, 0.25      , ..., 0.01694915, 0.        ,
        0.13559322],
       [0.        , 0.41176471, 0.        , ..., 0.05263158, 0.33333333,
        0.04444444],
       ...,
       [0.14285714, 0.        , 0.33333333, ..., 0.        , 0.        ,
        0.22857143],
       [0.09090909, 0.        , 0.18518519, ..., 0.01960784, 0.        ,
        0.09259259],
       [0.125     , 0.        , 0.25      , ..., 0.        , 0.        ,
        0.20512821]])

In [18]:
np.isclose(similarity_matrix_rdkit, similarity_matrix_scipy, rtol=1e-8, atol=1e-10).all()

True

# Calculate Tanimoto/Jaccard Similarity with Count Vectors

RDKit and Scipy disagree here since Scipy's jaccard is not identical to tanimoto for count vectors

## RDKit

In [19]:
fp_gen = AllChem.GetRDKitFPGenerator(minPath=1, maxPath=2, fpSize=2048)
def get_RDKit_fp(smi):
	mol = Chem.MolFromSmiles(smi)
	return fp_gen.GetCountFingerprint(mol)

In [20]:
similarity_matrix_rdkit = get_similarity_matrix(smiles_list1, smiles_list2, ncpus=1)
similarity_matrix_rdkit.shape

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    3.9s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    8.5s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   15.3s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:   24.1s


(600, 517)

In [21]:
similarity_matrix_rdkit

array([[0.11504425, 0.01149425, 0.15789474, ..., 0.04433498, 0.09756098,
        0.1797235 ],
       [0.37288136, 0.075     , 0.44444444, ..., 0.01666667, 0.        ,
        0.22058824],
       [0.        , 0.14893617, 0.        , ..., 0.04651163, 0.38095238,
        0.01515152],
       ...,
       [0.24      , 0.        , 0.35294118, ..., 0.        , 0.        ,
        0.39130435],
       [0.20689655, 0.        , 0.30120482, ..., 0.01010101, 0.        ,
        0.21008403],
       [0.2875    , 0.        , 0.359375  , ..., 0.        , 0.        ,
        0.25      ]])

## Scipy

In [22]:
fp_gen = AllChem.GetRDKitFPGenerator(minPath=1, maxPath=2, fpSize=2048)
def get_RDKit_fp(smi):
	mol = Chem.MolFromSmiles(smi)
	return fp_gen.GetCountFingerprintAsNumPy(mol)

In [23]:
fps1 = [get_RDKit_fp(smi) for smi in smiles_list1]
fps2 = [get_RDKit_fp(smi) for smi in smiles_list2]

In [24]:
similarity_matrix_scipy = 1 - distance.cdist(fps1, fps2, metric='jaccard')
similarity_matrix_scipy.shape

(600, 517)

In [25]:
similarity_matrix_scipy

array([[0.        , 0.        , 0.14285714, ..., 0.01960784, 0.        ,
        0.19607843],
       [0.        , 0.15789474, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.23529412, 0.        , ..., 0.        , 0.16666667,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11428571],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07407407],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

These results are not even close. Double check scipy's formula!

In [26]:
np.isclose(similarity_matrix_rdkit, similarity_matrix_scipy, rtol=1e-8, atol=1e-10).all()

False

# Calculate Dice Similarity with Bit Vectors
RDKit and Scipy agree here!

## RDKit

In [27]:
fp_gen = AllChem.GetRDKitFPGenerator(minPath=1, maxPath=2, fpSize=2048)
def get_RDKit_fp(smi):
	mol = Chem.MolFromSmiles(smi)
	return fp_gen.GetFingerprint(mol)

In [28]:
def get_sim(smi1, smiles_list):
	sim_results = np.zeros(len(smiles_list))
	for i, smi2 in enumerate(smiles_list):
		fp1 = get_RDKit_fp(smi1)
		fp2 = get_RDKit_fp(smi2)

		sim = DataStructs.DiceSimilarity(fp1, fp2)
		sim_results[i] = sim

	return sim_results

In [29]:
similarity_matrix_rdkit = get_similarity_matrix(smiles_list1, smiles_list2, ncpus=1)
similarity_matrix_rdkit.shape

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    3.9s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    8.5s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   15.4s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:   24.2s


(600, 517)

In [30]:
similarity_matrix_rdkit

array([[0.23076923, 0.1       , 0.44444444, ..., 0.17857143, 0.25      ,
        0.38095238],
       [0.35714286, 0.27272727, 0.4       , ..., 0.03333333, 0.        ,
        0.23880597],
       [0.        , 0.58333333, 0.        , ..., 0.1       , 0.5       ,
        0.08510638],
       ...,
       [0.25      , 0.        , 0.5       , ..., 0.        , 0.        ,
        0.37209302],
       [0.16666667, 0.        , 0.3125    , ..., 0.03846154, 0.        ,
        0.16949153],
       [0.22222222, 0.        , 0.4       , ..., 0.        , 0.        ,
        0.34042553]])

## Scipy

In [31]:
fps1 = [get_RDKit_fp(smi) for smi in smiles_list1]
fps2 = [get_RDKit_fp(smi) for smi in smiles_list2]

In [35]:
similarity_matrix_scipy = 1 - distance.cdist(fps1, fps2, metric='dice')
similarity_matrix_scipy.shape

(600, 517)

In [36]:
similarity_matrix_scipy

array([[0.23076923, 0.1       , 0.44444444, ..., 0.17857143, 0.25      ,
        0.38095238],
       [0.35714286, 0.27272727, 0.4       , ..., 0.03333333, 0.        ,
        0.23880597],
       [0.        , 0.58333333, 0.        , ..., 0.1       , 0.5       ,
        0.08510638],
       ...,
       [0.25      , 0.        , 0.5       , ..., 0.        , 0.        ,
        0.37209302],
       [0.16666667, 0.        , 0.3125    , ..., 0.03846154, 0.        ,
        0.16949153],
       [0.22222222, 0.        , 0.4       , ..., 0.        , 0.        ,
        0.34042553]])

In [37]:
np.isclose(similarity_matrix_rdkit, similarity_matrix_scipy, rtol=1e-8, atol=1e-10).all()

True

# Calculate Dice Similarity with Count Vectors
RDKit and Scipy disagree! 

In [38]:
fp_gen = AllChem.GetRDKitFPGenerator(minPath=1, maxPath=2, fpSize=2048)
def get_RDKit_fp(smi):
	mol = Chem.MolFromSmiles(smi)
	return fp_gen.GetCountFingerprint(mol)

In [39]:
similarity_matrix_rdkit = get_similarity_matrix(smiles_list1, smiles_list2, ncpus=1)
similarity_matrix_rdkit.shape

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    3.9s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    8.5s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:   15.3s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:   24.2s


(600, 517)

In [40]:
similarity_matrix_rdkit

array([[0.20634921, 0.02272727, 0.27272727, ..., 0.08490566, 0.17777778,
        0.3046875 ],
       [0.54320988, 0.13953488, 0.61538462, ..., 0.03278689, 0.        ,
        0.36144578],
       [0.        , 0.25925926, 0.        , ..., 0.08888889, 0.55172414,
        0.02985075],
       ...,
       [0.38709677, 0.        , 0.52173913, ..., 0.        , 0.        ,
        0.5625    ],
       [0.34285714, 0.        , 0.46296296, ..., 0.02      , 0.        ,
        0.34722222],
       [0.44660194, 0.        , 0.52873563, ..., 0.        , 0.        ,
        0.4       ]])

## Scipy

In [41]:
fp_gen = AllChem.GetRDKitFPGenerator(minPath=1, maxPath=2, fpSize=2048)
def get_RDKit_fp(smi):
	mol = Chem.MolFromSmiles(smi)
	return fp_gen.GetCountFingerprintAsNumPy(mol)

In [42]:
fps1 = [get_RDKit_fp(smi) for smi in smiles_list1]
fps2 = [get_RDKit_fp(smi) for smi in smiles_list2]

In [43]:
similarity_matrix_scipy = 1 - distance.cdist(fps1, fps2, metric='dice')
similarity_matrix_scipy.shape

(600, 517)

In [44]:
similarity_matrix_scipy

array([[2.52380952, 0.25      , 2.81818182, ..., 0.99056604, 1.68888889,
        2.0390625 ],
       [5.55555556, 0.13953488, 6.73846154, ..., 0.09836066, 0.        ,
        2.65060241],
       [0.        , 0.88888889, 0.        , ..., 0.62222222, 3.03448276,
        0.05970149],
       ...,
       [4.64516129, 0.        , 6.52173913, ..., 0.        , 0.        ,
        3.1875    ],
       [4.11428571, 0.        , 5.59259259, ..., 0.04      , 0.        ,
        2.02777778],
       [5.59223301, 0.        , 6.89655172, ..., 0.        , 0.        ,
        5.6       ]])

In [46]:
similarity_matrix_scipy.max()

36.67605633802817

In [47]:
np.isclose(similarity_matrix_rdkit, similarity_matrix_scipy, rtol=1e-8, atol=1e-10).all()

False